In [1]:
import os
import sys
sys.path.append(os.path.split(os.getcwd())[0])
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, precision_score, accuracy_score, f1_score
from sklearn.cross_validation import train_test_split
from collections import namedtuple
from pan_allele.helpers.pan_allele_data_helpers import *
from pan_allele.helpers.sequence_encoding import *
from pan_allele.helpers.amino_acid import *
from keras.models import Sequential, Graph
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.optimizers import SGD

Using Theano backend.
Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
def train_classII_data(model, peptides, Y_true, nb_epoch = 50):   
    for i in range(nb_epoch):
        X_final = []
        Y_final = []
        probs = []
        for idx, peptide in enumerate(peptides):
            if len(peptide) >=9:
                #get 9-mers from each peptide
                split_peptides = [peptide[pos:pos+9] for pos in range(0, len(peptide)-9 + 1) ]
                X = onehot(split_peptides, index_dict=amino_acid_letter_indices)
                #X = X.reshape(X.shape[0], X.shape[1]*X.shape[2]) 
                #predict binding strength values
                predictions = model.predict(X)
                #probability of 9-mer being a binding core
                #nromalize predictions to get probability of binding for each context
                prob = predictions/np.sum(predictions)
                X_final.extend(X)
                Y_final.extend([Y_true[idx]]*len(split_peptides))
                probs.extend(prob)
        X_final = np.array(X_final)
        Y_final = np.array(Y_final)
        probs = np.array(probs)
        model.fit(X_final,Y_final, nb_epoch=1, sample_weight=probs, verbose=1)
    #return model            
     

In [3]:
# Y_pred = []
#     Y_true_score = []
#     for idx, peptide in enumerate(peptides):
#           if len(peptide) >=9:
#                 split_peptides = [peptide[pos:pos+9] for pos in range(0, len(peptide)-9 + 1) ]
#                 ##Why not permanently reshape this?
#                 X = onehot(split_peptides, index_dict=amino_acid_letter_indices)
#                 #X = X.reshape(X.shape[0], X.shape[1]*X.shape[2]) 
#                 predictions = model.predict(X)
#                 Y_pred.append(np.mean(predictions))
#                 Y_true_score.append(Y_true[idx])

In [4]:
def get_test_set_metrics(model, peptides, Y_true):
    X = padded_indices(peptides, index_dict=amino_acid_letter_indices)
    Y_pred = model.predict({'input':X})['output']
    Y_binary = 50000**(1-Y_true)
    Y_binary = Y_binary<500
    
    Y_pred = np.array(Y_pred)
    Y_pred_binary = 50000**(1-Y_pred)
    Y_pred_binary = Y_pred_binary<500
    
    print "Training AUC:", roc_auc_score(Y_binary,Y_pred)
    print "Training Accuracy", accuracy_score(Y_binary, Y_pred_binary)
    print "F1 Score:", f1_score(Y_binary, Y_pred_binary)

In [5]:
def load_model(max_len):
    affinity_model = Sequential()
    affinity_model.add(Embedding(input_dim=22,output_dim=64,mask_zero=False ))
    affinity_model.add(LSTM(output_dim=64,input_dim=64, input_length=max_len ))
    affinity_model.add(Dense(1, input_dim=max_len, activation='sigmoid'))
    affinity_model.compile(loss='mse', optimizer='sgd')
    return affinity_model

In [6]:
def load_bRNN(max_len):
    model = Graph()
    model.add_input(name='input', input_shape=(max_len,), dtype=int)
    model.add_node(Embedding(22, 128, input_length=max_len, mask_zero=True),
                   name='embedding', input='input')
    model.add_node(LSTM(64), name='forward', input='embedding')
    model.add_node(LSTM(64, go_backwards=True), name='backward', input='embedding')
    model.add_node(Dense(1, activation='sigmoid'), name='sigmoid', inputs=['forward', 'backward'])
    model.add_output(name='output', input='sigmoid')
    model.compile('rmsprop', {'output': 'mse'})
    return model

In [7]:
def train_LSTM(model, peptides, Y_true, nb_epoch = 50):    
    X = padded_indices(peptides, index_dict=amino_acid_letter_indices)
    model = load_bRNN(max_len = X.shape[1])
    model.fit({'input':X,'output':Y_true}, nb_epoch=nb_epoch,verbose=1)
    return model

In [8]:
allele_groups, df = load_binding_data('bdata.classii.2010.csv', max_ic50=50000)
allele_list = [ 'DRB10101', 'DRB10301', 'DRB10401', 'DRB10404', 'DRB10405',
                'DRB10701', 'DRB10802', 'DRB10901', 'DRB11101', 'DRB11302', 
                'DRB11501', 'DRB30101', 'DRB40101', 'DRB50101']
allele_list = ['DRB10101']


In [9]:
Y_true, peptides, ic50 = allele_groups['DRB10101']
peptide_train, peptide_test, Y_train, Y_test = train_test_split(peptides, Y_true, test_size = 0.2)

In [10]:
model = train_LSTM(None, peptide_train, Y_train, nb_epoch=5)

Epoch 1/5
5788/5788 [==============================] - 12s - loss: 0.0750    
Epoch 2/5
5788/5788 [==============================] - 12s - loss: 0.0670    
Epoch 3/5
5788/5788 [==============================] - 13s - loss: 0.0582    
Epoch 4/5
5788/5788 [==============================] - 13s - loss: 0.0561    
Epoch 5/5
5788/5788 [==============================] - 13s - loss: 0.0542    


In [20]:
get_test_set_metrics(model, peptide_test, Y_test)

Training AUC: 0.813181820002
Training Accuracy 0.724948168625
F1 Score: 0.802775024777


In [20]:
for layer in  model.nodes.values():
    print layer.output_shape

(None, 37, 128)
(None, 64)
(None, 64)
(None, 1)
